In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import joblib
import re
import sys
sys.path.insert(0, '../')
import numpy as np

# Datasets
from aif360.datasets import MEPSDataset19
from aif360.datasets import MEPSDataset20
from aif360.datasets import MEPSDataset21
from aif360.datasets import GermanDataset
# Fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric

# Explainers
from aif360.explainers import MetricTextExplainer

# Scalers
from sklearn.preprocessing import StandardScaler


# Bias mitigation techniques
from aif360.algorithms.preprocessing import Reweighing,DisparateImpactRemover
from aif360.algorithms.preprocessing import LFR
from aif360.algorithms.preprocessing import OptimPreproc
from sklearn.model_selection import train_test_split

In [ ]:
dataset_orig_panel19 = MEPSDataset19()

In [ ]:
dataset_orig_panel19_train = MEPSDataset19()

In [ ]:
dataset_orig_panel19_train.features

In [ ]:
sens_ind = 0
sens_attr = dataset_orig_panel19_train.protected_attribute_names[sens_ind]
unprivileged_groups = [{sens_attr: v} for v in
                    dataset_orig_panel19_train.unprivileged_protected_attributes[sens_ind]]
privileged_groups = [{sens_attr: v} for v in
                    dataset_orig_panel19_train.privileged_protected_attributes[sens_ind]]

In [ ]:
sens_attr

In [ ]:
privileged_groups

In [ ]:
unprivileged_groups

In [ ]:
dataset_orig_panel19_train

In [ ]:
metric_orig_panel19_train = BinaryLabelDatasetMetric(
        dataset_orig_panel19_train,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
# Prepare data
X = dataset_orig_panel19_train.features
y = dataset_orig_panel19_train.labels.ravel()

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train logistic regression model
model = LogisticRegression(random_state=42)
model.fit(X_train_scaled, y_train)

# Predict probabilities on the same scaled training data
train_probabilities = model.predict_proba(X_train_scaled)[:, 1]

# Calculation of discrimination index without modifying dataset structure
sens_attr_index = dataset_orig_panel19_train.feature_names.index('RACE')

def calculate_discrimination(X, probabilities, sens_attr_index, unprivileged_val, privileged_val):
    # Filter by sensitive attribute for unprivileged and privileged groups
    unpriv_indices = X[:, sens_attr_index] == unprivileged_val
    priv_indices = X[:, sens_attr_index] == privileged_val
    
    # Calculate mean probabilities for both groups
    mean_prob_unpriv = probabilities[unpriv_indices].mean()
    mean_prob_priv = probabilities[priv_indices].mean()
    
    # Discrimination index
    discrimination = mean_prob_priv - mean_prob_unpriv
    return discrimination

# Define unprivileged and privileged values
unprivileged_val = 0.0
privileged_val = 1.0

# Compute discrimination
discrimination_index = calculate_discrimination(X_train, train_probabilities, sens_attr_index, unprivileged_val, privileged_val)
print("Discrimination Index: {:.4f}".format(discrimination_index))


In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

def custom_loss(lambda_val=0.001):
    def loss(y_true, y_pred):
        # Extract predictions and sensitive attributes from y_pred
        predictions = y_pred[:, 0]
        sensitive_attr = y_pred[:, 1]

        # Debug prints to check outputs
        # tf.print("Predictions sample:", predictions[:10])
        # tf.print("Sensitive Attr sample:", sensitive_attr[:10])

        # Standard binary crossentropy loss
        standard_loss = BinaryCrossentropy(from_logits=True)(y_true, predictions)
        
        # Determine thresholds to convert sensitive attributes to binary
        # Assuming the negative and positive classes are split around zero
        threshold = 0
        mask_unpriv = K.cast(sensitive_attr <= threshold, 'float32')
        mask_priv = K.cast(sensitive_attr > threshold, 'float32')

        epsilon = 1e-8
        sum_unpriv = K.sum(mask_unpriv)
        sum_priv = K.sum(mask_priv)

        # # Debug prints for mask sums
        # tf.print("Sum unprivileged:", sum_unpriv)
        # tf.print("Sum privileged:", sum_priv)

        prob_unpriv = K.sum(predictions * mask_unpriv) / (sum_unpriv + epsilon)
        prob_priv = K.sum(predictions * mask_priv) / (sum_priv + epsilon)

        # Discrimination as the squared difference in probabilities
        discrimination = K.square(prob_priv - prob_unpriv)

        # # Debug print
        # tf.print("Discrimination:", discrimination)

        # Total loss with discrimination penalty
        return standard_loss + lambda_val * discrimination
    
    return loss



# Model parameters
input_size = X_train_scaled.shape[1]  # Number of features
sensitive_index = dataset_orig_panel19_train.feature_names.index('RACE')

# Input layers
inputs = Input(shape=(input_size,))
sensitive_inputs = Input(shape=(1,))

# Network architecture
x = Dense(64, activation='relu')(inputs)
outputs = Dense(1, activation='sigmoid')(x)
combined_outputs = tf.keras.layers.concatenate([outputs, sensitive_inputs])

model = Model(inputs=[inputs, sensitive_inputs], outputs=combined_outputs)

# Compile the model with the custom loss function
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=custom_loss(lambda_val = 0.01),
              metrics=['accuracy'])

# Prepare data with sensitive attribute
X_train_with_sensitive = [X_train_scaled, X_train_scaled[:, sensitive_index]]

# Train the model
history1_nd = model.fit(X_train_with_sensitive, y_train, epochs=100, batch_size=32, validation_split=0.2)


In [ ]:

# Model parameters
input_size = X_train_scaled.shape[1]  # Number of features
sensitive_index = dataset_orig_panel19_train.feature_names.index('RACE')

# Input layers
inputs = Input(shape=(input_size,))
sensitive_inputs = Input(shape=(1,))

# Network architecture
x = Dense(64, activation='relu')(inputs)
outputs = Dense(1, activation='sigmoid')(x)

# No need to concatenate outputs and sensitive inputs for the loss calculation
# Separate outputs for predictions and sensitive attributes
# Since we're using binary_crossentropy, we only need the main outputs
model = Model(inputs=[inputs, sensitive_inputs], outputs=outputs)

# Compile the model with binary crossentropy
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Prepare data with sensitive attribute
X_train_with_sensitive = [X_train_scaled, X_train_scaled[:, sensitive_index]]

# Train the model
history2_nd = model.fit(X_train_with_sensitive, y_train, epochs=100, batch_size=32, validation_split=0.2)
